# Richter's Predictor: Modeling Earthquake Damage

Based on aspects of building location and construction, the goal is to predict the level of damage to buildings caused by the 2015 Gorkha earthquake in Nepal.

The data was collected through surveys by [Kathmandu Living Labs](http://www.kathmandulivinglabs.org) and the [Central Bureau of Statistics](https://cbs.gov.np), which works under the National Planning Commission Secretariat of Nepal. This survey is **one of the largest post-disaster datasets ever collected**, containing valuable information on earthquake impacts, household conditions, and socio-economic-demographic statistics.

In [1]:
import pandas as pd
import numpy as np
import os
import re

import string 
import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# ! pip install imbalanced-learn
import imblearn
# print(imblearn.__version__)
from imblearn.over_sampling import SMOTE

In [3]:
import sklearn

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, StratifiedKFold, cross_val_score

from sklearn.pipeline import Pipeline

from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, roc_auc_score, roc_curve, confusion_matrix

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import RFECV, SelectFromModel

import multiprocessing

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [5]:
# Load data set
train = pd.read_csv(os.path.join('', 'Richters_Predictor_Modeling_Earthquake_Damage_-_Train_Values.csv'))

In [6]:
test = pd.read_csv(os.path.join('', 'Richters_Predictor_Modeling_Earthquake_Damage_-_Test_Values.csv'))

In [7]:
labels = pd.read_csv(os.path.join('', 'Richters_Predictor_Modeling_Earthquake_Damage_-_Train_Labels.csv'))

In [8]:
labels.damage_grade.value_counts()/len(labels.damage_grade)

2    0.568912
3    0.334680
1    0.096408
Name: damage_grade, dtype: float64

**Remarks:**

- According to these results, we can say that there are 56.89% of the building suffered a medium amount of damage, 33.46% of the building were almost completely distructed and 9.64% of the building suffered a low damage.

- Class "low damage" is imbalanced, and might need upsampling. Another option to deal with the imbalance is to choose an appropriate metric, like F1 score or AUC.

In [9]:
train.dtypes.value_counts()

int64     31
object     8
dtype: int64

In [10]:
print('Object data types:\n')
#we'll use the function later, without wanting to print anything
def get_obj(train, p = False):
    obj_types = []
    for column in train.columns:
        if train[column].dtype == 'object': 
            if p: print(column)
            obj_types.append(column)
    return obj_types

Object data types:



In [11]:
obj_types = get_obj(train, True)

land_surface_condition
foundation_type
roof_type
ground_floor_type
other_floor_type
position
plan_configuration
legal_ownership_status


In [12]:
def transform_to_int(train, obj_types):
    #Assign dictionaries with current values and replacements for each column
    d_lsc = {'n':0, 'o':1, 't':2}
    d_ft = {'h':0, 'i':1, 'r':2, 'u':3, 'w':4}
    d_rt = {'n':0, 'q':1, 'x':2}
    d_gft = {'f':0, 'm':1, 'v':2, 'x':3, 'z':4}
    d_oft = {'j':0, 'q':1, 's':2, 'x':3}
    d_pos = {'j':0, 'o':1, 's':2, 't':3}
    d_pc = {'a':0, 'c':1, 'd':2, 'f':3, 'm':4, 'n':5, 'o':6, 'q':7, 's':8, 'u':9}
    d_los = {'a':0, 'r':1, 'v':2, 'w':3}
    #Each positional index in replacements corresponds to the column in obj_types
    replacements = [d_lsc, d_ft, d_rt, d_gft, d_oft, d_pos, d_pc, d_los]

    #Replace using lambda Series.map(lambda)
    for i,col in enumerate(obj_types):
        train[col] = train[col].map(lambda a: replacements[i][a]).astype('int64')

In [13]:
transform_to_int(train, obj_types)

In [14]:
train.dtypes.value_counts()

int64    39
dtype: int64

In [15]:
train.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,2,2,0,0,1,3,2,1,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,1,2,0,3,1,2,2,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,2,2,0,0,3,3,2,0,1,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,2,2,0,0,3,2,2,0,1,0,0,0,0,1,1,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,2,2,0,0,3,2,2,1,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0


***
**Data Splitting**
***

In [16]:
y = labels.pop('damage_grade')  
x = train.drop(["building_id"],axis=1)

In [17]:
print('Original dataset shape:', x.shape)
print('Original labelset shape:', y.shape)

Original dataset shape: (260601, 38)
Original labelset shape: (260601,)


In [18]:
# keep the same random state for reproducibility
RANDOM_STATE = 12
TRAIN_TEST_SPLIT_SIZE = .1

In [19]:
# stratify on damage_grade
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = TRAIN_TEST_SPLIT_SIZE, 
                                                    stratify = y,
                                                    random_state = RANDOM_STATE)

In [20]:
print('Training dataset shape:', x_train.shape)
print('Training labelset shape:', y_train.shape)

Training dataset shape: (234540, 38)
Training labelset shape: (234540,)


In [21]:
print('Test labelset shape:', y_test.shape)
print('Test dataset shape:', x_test.shape)

Test labelset shape: (26061,)
Test dataset shape: (26061, 38)


Scaling is typically done to normalize data so that priority is not given to a particular feature. The role of scaling is mostly important in algorithms that are distance based and require Euclidean Distance. Random Forest is a tree-based model and hence does not require feature scaling.

***
**Feature selection by feature importance of random forest classifier**
***

In [43]:
import joblib
# load, no need to initialize the loaded model
clf = joblib.load("./rf.joblib")

In [44]:
sel = SelectFromModel(clf)
sel.fit(x_train, y_train)
sel.get_support()

array([ True,  True,  True, False,  True,  True,  True, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [45]:
x_train.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [46]:
features = x_train.columns[sel.get_support()]
features

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'age',
       'area_percentage', 'height_percentage', 'foundation_type', 'roof_type'],
      dtype='object')

In [47]:
np.mean(sel.estimator_.feature_importances_)

0.026315789473684216

In [48]:
sel.estimator_.feature_importances_

array([1.93379371e-01, 1.42012206e-01, 1.32702566e-01, 2.06482662e-02,
       8.10418179e-02, 7.71536347e-02, 4.90295682e-02, 1.80564254e-02,
       3.25541605e-02, 2.81021526e-02, 2.07550747e-02, 1.97617015e-02,
       1.85951126e-02, 1.04573406e-02, 1.00510791e-02, 2.52429449e-02,
       7.85369264e-03, 3.89550364e-03, 7.97674646e-03, 1.30758940e-02,
       1.25919532e-02, 7.46085099e-03, 6.49208467e-03, 4.56660675e-03,
       4.11776004e-03, 9.15139792e-03, 2.28645559e-02, 7.68989627e-03,
       5.27906840e-03, 3.69521203e-03, 1.43548940e-03, 2.77533546e-04,
       1.27388927e-04, 4.26195002e-04, 6.99466997e-05, 3.94744527e-05,
       4.29741370e-05, 1.32635259e-03])

***
**Recursive Feature Elimination (RFE)**
***

In [49]:
def run_randomForest(x_train, x_test, y_train, y_test, clf_rf):
#     clf = RandomForestClassifier(n_estimators=600, random_state=0, n_jobs=1)
    clf_rf.fit(x_train, y_train)
    y_pred = clf_rf.predict(x_test)
    score = f1_score(y_test, y_pred, average='micro')
    print(f'{score:.4f}')
    

In [58]:
from sklearn.feature_selection import RFE
sel = RFE(clf, n_features_to_select = 10)
sel.fit(x_train, y_train)

RFE(estimator=RandomForestClassifier(max_depth=110, max_features='sqrt',
                                     min_samples_split=10, n_estimators=500),
    n_features_to_select=10)

In [59]:
sel.get_support()

array([ True,  True,  True, False,  True,  True,  True, False,  True,
        True, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [60]:
features = x_train.columns[sel.get_support()]
features

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'age',
       'area_percentage', 'height_percentage', 'foundation_type', 'roof_type',
       'has_superstructure_mud_mortar_stone', 'count_families'],
      dtype='object')

In [61]:
x_train_rfe = sel.transform(x_train)
x_test_rfe = sel.transform(x_test)

In [62]:
%%time
run_randomForest(x_train_rfe, x_test_rfe, y_train, y_test, clf)

0.7275
CPU times: user 3min 38s, sys: 4.99 s, total: 3min 43s
Wall time: 3min 56s


In [64]:
# importance_rf = pd.DataFrame({"Features":x.columns, "Importance_RF":sel.feature_importances_}).sort_values(by='Importance_RF', ascending = False).head(15)

# RF_styler = importance_rf.style.set_table_attributes("style='display:inline'").set_caption('Top 15 Random Forest importance')

# from IPython.display import display_html 
# display_html(RF_styler._repr_html_(), raw=True)

In [66]:
import joblib
# save
joblib.dump(clf, "./rf_rfe.joblib")

['./rf_rfe.joblib']

In [67]:
# load, no need to initialize the loaded model
loaded_rf = joblib.load("./rf_rfe.joblib")

In [68]:
transform_to_int(test, obj_types)

In [69]:
test = test.drop(["building_id"],axis=1)

In [71]:
predictions = clf.predict(test)

In [ ]:
submission_format = pd.read_csv('Richters_Predictor_Modeling_Earthquake_Damage_-_Submission_Format.csv', index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.head()

In [ ]:
my_submission.to_csv('submission.csv')

In [ ]:
!head submission.csv